## Neural Network

In [ ]:
#

import numpy as np

import matplotlib.pyplot as plt

import copy as copy

from neuralnetwork.neural_network_module import NeuralNetwork, Layer
from neuralnetwork.neural_network_module import train_nn_classification

np.random.seed(101)

### Create and plot data series

In [ ]:
# Create data for sinusoid

N = int(2e3) # number of samples

xx = 2*(np.random.rand(N,2) - 0.5)

yy = np.zeros((N,3), dtype=int)



for kx in np.arange(-1,1.25,0.25):
    idx = abs(xx[:,0] - kx) <= 0.1/2
    # print(idx)
    yy[idx,0] = 1
    idx = abs(xx[:,0] - kx) <= 0.05/2
    yy[idx,0] = 2
for ky in np.arange(-1,1.25,0.25):
    idx = abs(xx[:,1] - ky) <= 0.1/2
    yy[idx,1] = 1
    idx = abs(xx[:,1] - ky) <= 0.05/2
    yy[idx,1] = 2
for kx in np.arange(0,0.25,0.25):
    for ky in np.arange(0,0.25,0.25):
        # idx = np.where(((xx - kx)**2 <= 0.01**2) & ((yy - ky)**2 <= 0.01**2))
        # yy[idx,2] = 1
        # idx = np.where(((xx - kx)**2 <= 0.025**2) & ((yy - ky)**2 <= 0.025**2))
        # yy[idx,2] = 2

        idx = np.where(((xx[:,0]- kx)**2 + (xx[:,1] - ky)**2 <= 0.5**2))
        yy[idx,2] = 1
        idx = np.where(((xx[:,0] - kx)**2 + (xx[:,1] - ky)**2 <= 0.25**2))
        yy[idx,2] = 2

xp, yp = np.meshgrid(sorted(xx[:,0]), sorted(xx[:,1]))
zxp = np.zeros(xp.shape)
zyp = np.zeros(xp.shape)
zxyp = np.zeros(xp.shape)
for kx in np.arange(0,0.25,0.25):
    idx = abs(xp - kx) <= 0.1
    zxp[idx] = 1
    idx = abs(xp - kx) <= 0.05
    zxp[idx] = 2
for ky in np.arange(0,0.25,0.25):
    idx = abs(yp - ky) <= 0.1
    zyp[idx] = 1
    idx = abs(yp - ky) <= 0.05
    zyp[idx] = 2
for kx in np.arange(-1,1.5,0.5):
    for ky in np.arange(-1,1.5,0.5):
        # idx = np.where(((xp - kx)**2 <= 0.25**2) & ((yp - ky)**2 <= 0.25**2))
        # zxyp[idx] = 1
        # idx = np.where(((xp - kx)**2 <= 0.5**2) & ((yp - ky)**2 <= 0.5**2))
        # zxyp[idx] = 2

        idx = np.where(((xx[:,0]- kx)**2 + (xx[:,1] - ky)**2 <= 0.2**2))
        yy[idx,2] = 1
        idx = np.where(((xx[:,0] - kx)**2 + (xx[:,1] - ky)**2 <= 0.1**2))
        yy[idx,2] = 2

        idx = np.where(((xp - kx)**2 + (yp - ky)**2 <= 0.2**2))
        zxyp[idx] = 1
        idx = np.where(((xp - kx)**2 + (yp - ky)**2 <= 0.1**2))
        zxyp[idx] = 2


plt.figure(figsize=(24,6))
plt.subplot(1,3,1)
plt.imshow(zxp, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,3,2)
plt.imshow(zyp, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,3,3)
plt.imshow(zxyp, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# print(xx)
# print(yy)

### Split data into training and test sets

In [ ]:
#

train_test_split = 0.8

# split into train and test sets
idx_train = range(0,int(train_test_split*xx.shape[0])) # training set indexes
X_train = xx[idx_train,:]
Y_train = yy[idx_train,:]

idx_test = range(int(train_test_split*xx.shape[0]),xx.shape[0]) # test set indexes
X_test = xx[idx_test,:]
Y_test = yy[idx_test,:]

### Create functions for training neural network and assessing its performance

In [ ]:
#

def one_hot(Y, num_cols:int=2):
    # one_hot_Y = np.zeros((Y.shape[0], Y.max()+1))
    one_hot_Y = np.zeros((Y.shape[0], num_cols))
    one_hot_Y[np.arange(0, Y.shape[0]), Y] = 1
    return one_hot_Y

def get_predictions(A2):
    return np.argmax(A2, 1).reshape(-1,1)

def get_accuracy(Y_pred, Y):
    # print("Values:", Y.T)
    # print("Predictions:", Y_pred.T)    
    return np.sum(Y_pred == Y)/Y.shape[0]

def train_nn(NN: "NeuralNetwork", X_train, Y_train, alpha, iterations=100, intervals=10, X_test=None, Y_test=None):
    train_accuracy_list = []
    test_accuracy_list = []
    NN_list = []
    epoch_list = []
    for k1 in range(iterations):
        NN._compute_output(X_train)
        NN._train_step(X_train, one_hot(Y_train[:,0], 3), alpha)
        Y_pred = get_predictions(NN._compute_output(X_train))
        if k1 == 0 or (k1+1) % intervals == 0:
            print("")
            Y_pred_train = get_predictions(NN._compute_output(X_train))
            train_accuracy = get_accuracy(Y_pred_train, Y_train)
            train_accuracy_list.append(train_accuracy)
            if X_test is None and Y_test is None:
                print(f"Iteration: {k1} | Train accuracy: {train_accuracy:0.4f}")
            if X_test is not None and Y_test is not None:
                Y_pred_test = get_predictions(NN._compute_output(X_test))
                test_accuracy = get_accuracy(Y_pred_test, Y_test)
                test_accuracy_list.append(test_accuracy)
                print(f"Iteration: {k1} | Train accuracy: {train_accuracy:0.4f} | Test accuracy: {test_accuracy:0.4f}")
            print(f"Train Targets:     {[x for x in Y_train[0:24,0]]}")
            print(f"Train Predictions: {[x for x in Y_pred[0:24,0]]}")

            epoch_list.append(k1)
            NN_list.append(copy.deepcopy(NN))

    return NN_list, train_accuracy_list, test_accuracy_list, epoch_list


### Create and train neural network and analyze its performance

In [ ]:
#

nn = NeuralNetwork()

layer_list = []
# layer_list.append(Layer())

nn._add_layers([Layer(2, None)])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(16, "linear")])
nn._add_layers([Layer(3, "softmax")])

NN_list, train_accuracy, test_accuracy, epochs = train_nn_classification(nn, X_train, Y_train[:,[2]], 0.020, 50000, 1000, X_test, Y_test[:,[2]])

print("")
print("Training Complete")
print(f"Train accuracy: {get_accuracy(get_predictions(nn._compute_output(X_train)), Y_train[:,[2]]):0.4f}")
print(f"Test accuracy: {get_accuracy(get_predictions(nn._compute_output(X_test)), Y_test[:,[2]]):0.4f}")


In [ ]:
#

plt.figure(figsize=(12,6))
plt.plot(epoch_list, train_accuracy_list, 'b.', label="Train accuracy")
plt.plot(epoch_list, test_accuracy_list, 'g.', label="Test accuracy")
plt.xticks(epoch_list)
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
xp, yp = np.meshgrid(sorted(X_train[:,0]), sorted(X_train[:,1]))
zxp_train = np.zeros(xp.shape)
zyp_train = np.zeros(xp.shape)
zxyp_train = np.zeros(xp.shape)

zxyp_train = get_predictions(nn._compute_output(np.hstack((xp.reshape(-1,1), yp.reshape(-1,1)))))
zxyp_train = zxyp_train.reshape(xp.shape)

xp, yp = np.meshgrid(sorted(X_test[:,0]), sorted(X_test[:,1]))

zxyp_test = get_predictions(nn._compute_output(np.hstack((xp.reshape(-1,1), yp.reshape(-1,1)))))
zxyp_test = zxyp_test.reshape(xp.shape)

plt.figure(figsize=(18,18))
plt.subplot(1,3,1)
plt.imshow(zxyp, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,3,2)
plt.imshow(zxyp_train, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,3,3)
plt.imshow(zxyp_test, cmap=plt.get_cmap('gray'), extent=[-1, 1, -1, 1])
plt.xlabel("x")
plt.ylabel("y")
plt.show()